In [3]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

from utils import expected
np.random.seed(1)

def change(A, B, c_plus, c_minus, u_plus, u_minus, prob=0.4):
    
    # Working with array-like structures
    A = np.asarray(A)
    B = np.asarray(B)

    # Calculated predicted change of both sets of samples
    delta_A = expected(A, c_plus, c_minus)
    delta_B = expected(B, c_plus, c_minus)

    A_matrix = np.repeat(A, A.shape[0]).reshape((A.shape[0], A.shape[0]))
    B_matrix = np.repeat(B, B.shape[0]).reshape((B.shape[0], B.shape[0]))
    delta_A_matrix = np.repeat(delta_A, A.shape[0]).reshape((A.shape[0], A.shape[0]))
    delta_B_matrix = np.repeat(delta_B, B.shape[0]).reshape((B.shape[0], B.shape[0]))

    jitter = np.random.choice([1e-8, -1e-8], size=A.shape, p=[0.4, 0.6])
    noise = (A_matrix + delta_A_matrix) == A_matrix.T
    A_matrix = np.where(noise, A_matrix + jitter, A_matrix)
    noise = (B_matrix + delta_B_matrix) == B_matrix.T
    B_matrix = np.where(noise, B_matrix + jitter, B_matrix)
    
    A_matrix = np.where(A_matrix + delta_A_matrix > A_matrix.T, A_matrix + delta_A_matrix, A_matrix)
    B_matrix = np.where(B_matrix + delta_B_matrix > B_matrix.T, B_matrix + delta_B_matrix, B_matrix)

    mean_A = np.mean(A_matrix, axis=0)
    mean_B = np.mean(B_matrix, axis=0)
    mean_A = np.repeat(mean_A, mean_A.shape[0]).reshape((mean_A.shape[0], mean_A.shape[0]))
    mean_B = np.repeat(mean_B, mean_B.shape[0]).reshape((mean_B.shape[0], mean_B.shape[0])).T

    util_A = np.sum(expected(A_matrix, u_plus, u_minus), axis=0)
    util_B = np.sum(expected(B_matrix, u_plus, u_minus), axis=0)
    util_A = np.repeat(util_A, util_A.shape[0]).reshape((util_A.shape[0], util_A.shape[0]))
    util_B = np.repeat(util_B, util_B.shape[0]).reshape((util_B.shape[0], util_B.shape[0]))

    return mean_A, mean_B, util_A, util_B

def fair_opt_step(A, B, u_plus, u_minus, c_plus, c_minus, alpha):
    
    # Working with array-like structures
    A = np.array(A)
    B = np.array(B)
    np.random.seed(1)
    prob = 0.4

    # Calculate population percentages
    w_a = len(A) / (len(A) + len(B))
    w_b = 1 - w_a

    # Build meshgrid for threshold pairs
    mean_A, mean_B = np.meshgrid(A, B, indexing='ij')
    util_A, util_B = np.meshgrid(A, B, indexing='ij')

    # Matrix of every threshold combination
    mean_A, mean_B, util_A, util_B = change(A, B, c_plus, c_minus, u_plus, u_minus, prob)

    # Calculate fairness difference at each pair
    fairness_diff = np.abs(mean_A - mean_B)

    # Calculate weighted total utility for each pair
    total_util = w_a * util_A + w_b * util_B

    # Mask utilities violating fairness constraint

    total_util[fairness_diff > alpha] = -np.inf

    flat_idx = np.argmax(total_util)
    i_idx, j_idx = np.unravel_index(flat_idx, total_util.shape)

    opt_A = A[i_idx]  # index along A dimension
    opt_B = B[j_idx]  # index along B dimension

    updated_samples = (mean_A[i_idx, j_idx], mean_B[i_idx, j_idx])  # updated after change

    max_util = total_util[i_idx, j_idx]

    return (opt_A, opt_B, max_util, updated_samples, total_util[i_idx][j_idx])


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from utils import opt_step, test_fair_opt_step 

# Distribution parameters
mean_a, std_a = 0.5, 1
mean_b, std_b = 0, 1

# Domain of alpha values to test
alpha_min, alpha_max = 0, 2
alphas = np.linspace(alpha_min, alpha_max, num=100)

# Sample size of each distribution
sample_size = 1000

# Generate samples from normal distributions
np.random.seed(1)
a = np.random.normal(mean_a, std_a, sample_size)
b = np.random.normal(mean_b, std_b, sample_size)

# Store results
x = []
fair_mean_A = []
fair_mean_B = []
fair_threshold_A = []
fair_threshold_B = []
total_util = []

# Unconstrained optimal means
opt_result_A = opt_step(a, 1, -1, 1, -1)[1]
opt_result_B = opt_step(b, 1, -1, 1, -1)[1]
y3 = np.mean(opt_result_A)
y4 = np.mean(opt_result_B)

# Run fair optimization over a range of alpha values
for alpha in alphas:
    results = test_fair_opt_step(a, b, 1, -1, 1, -1, alpha)
    if results is not None:
        A, B = results[3]
        if np.abs(A - B) > alpha:
            # Only store results if the fairness constraint is satisfied
            continue
        x.append(alpha)
        fair_mean_A.append(A)
        fair_mean_B.append(B)
        fair_threshold_A.append(results[0])
        fair_threshold_B.append(results[1])
        total_util.append(((results[4])*0.01))

def plot_fairness(show_means=True, show_thresholds=True, show_utility=True):
    plt.figure(figsize=(10, 6))

    if show_means:
        plt.plot(x, fair_mean_A, label="Fair μ(A)'", color='red')
        plt.plot(x, fair_mean_B, label="Fair μ(B)'", color='blue')
        plt.axhline(y3, color='red', linestyle='--', label='Optimal μ(A)\' (Unconstrained)')
        plt.axhline(y4, color='blue', linestyle='--', label='Optimal μ(B)\' (Unconstrained)')
        plt.plot(0, mean_a, 'ro', label='Initial μ(A)')
        plt.plot(0, mean_b, 'bo', label='Initial μ(B)')

    if show_thresholds:
        plt.plot(x, fair_threshold_A, label="Fair Threshold (A)", color='orange')
        plt.plot(x, fair_threshold_B, label="Fair Threshold (B)", color='purple')

    if show_utility:
        plt.plot(x, total_util, label='Total Utility', color='green')

    plt.title("Fair vs. Optimal Means under Varying Fairness Constraint α")
    plt.xlabel("α (Fairness Threshold)")
    plt.ylabel("Metric Value")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Toggle widgets
means_toggle = widgets.Checkbox(value=True, description='Show Means')
thresholds_toggle = widgets.Checkbox(value=True, description='Show Thresholds')
utility_toggle = widgets.Checkbox(value=True, description='Show Utility')

ui = widgets.HBox([means_toggle, thresholds_toggle, utility_toggle])
out = widgets.interactive_output(plot_fairness, {
    'show_means': means_toggle,
    'show_thresholds': thresholds_toggle,
    'show_utility': utility_toggle
})

display(ui, out)

Output()

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from utils import opt_step, test_fair_opt_step 

# Distribution parameters
mean_a, std_a = 0.5, 1
mean_b, std_b = 0, 1

# Domain of alpha values to test
alpha_min, alpha_max = 0, 2
alphas = np.linspace(alpha_min, alpha_max, num=100)

# Sample size of each distribution
sample_size = 1000

# Generate samples from normal distributions
np.random.seed(1)
a = np.random.normal(mean_a, std_a, sample_size)
b = np.random.normal(mean_b, std_b, sample_size)

# Store results
x = []
fair_mean_A = []
fair_mean_B = []
fair_threshold_A = []
fair_threshold_B = []
total_util = []

# Unconstrained optimal means
opt_result_A = opt_step(a, 1, -1, 1, -1)[1]
opt_result_B = opt_step(b, 1, -1, 1, -1)[1]
y3 = np.mean(opt_result_A)
y4 = np.mean(opt_result_B)

# Run fair optimization over a range of alpha values
for alpha in alphas:
    results = fair_opt_step(a, b, 1, -1, 1, -1, alpha)
    if results is not None:
        A, B = results[3]
        if np.abs(A - B) > alpha:
            # Only store results if the fairness constraint is satisfied
            continue
        x.append(alpha)
        fair_mean_A.append(A)
        fair_mean_B.append(B)
        fair_threshold_A.append(results[0])
        fair_threshold_B.append(results[1])
        total_util.append(((results[4])*0.01))

def plot_fairness(show_means=True, show_thresholds=True, show_utility=True):
    plt.figure(figsize=(10, 6))

    if show_means:
        plt.plot(x, fair_mean_A, label="Fair μ(A)'", color='red')
        plt.plot(x, fair_mean_B, label="Fair μ(B)'", color='blue')
        plt.axhline(y3, color='red', linestyle='--', label='Optimal μ(A)\' (Unconstrained)')
        plt.axhline(y4, color='blue', linestyle='--', label='Optimal μ(B)\' (Unconstrained)')
        plt.plot(0, mean_a, 'ro', label='Initial μ(A)')
        plt.plot(0, mean_b, 'bo', label='Initial μ(B)')

    if show_thresholds:
        plt.plot(x, fair_threshold_A, label="Fair Threshold (A)", color='orange')
        plt.plot(x, fair_threshold_B, label="Fair Threshold (B)", color='purple')

    if show_utility:
        plt.plot(x, total_util, label='Total Utility', color='green')

    plt.title("Fair vs. Optimal Means under Varying Fairness Constraint α")
    plt.xlabel("α (Fairness Threshold)")
    plt.ylabel("Metric Value")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Toggle widgets
means_toggle = widgets.Checkbox(value=True, description='Show Means')
thresholds_toggle = widgets.Checkbox(value=True, description='Show Thresholds')
utility_toggle = widgets.Checkbox(value=True, description='Show Utility')

ui = widgets.HBox([means_toggle, thresholds_toggle, utility_toggle])
out = widgets.interactive_output(plot_fairness, {
    'show_means': means_toggle,
    'show_thresholds': thresholds_toggle,
    'show_utility': utility_toggle
})

display(ui, out)

Output()